Files are too big for this approach.

Also these are the most recent crawl, so won't have urls that have since disappeared (like government agencies!).

In [ ]:
import requests
import shutil
import pandas as pd
import gzip
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

s = requests.Session()
retries = Retry(total=10, backoff_factor=1, status_forcelist=[ 502, 503, 504 ])
s.mount('https://', HTTPAdapter(max_retries=retries))
s.mount('http://', HTTPAdapter(max_retries=retries))

In [ ]:
# Get latest index
response = requests.get('https://index.commoncrawl.org/collinfo.json')
current_idx = response.json()[0]

In [ ]:
current_idx

In [ ]:
cluster_url = f'https://commoncrawl.s3.amazonaws.com/cc-index/collections/{current_idx["id"]}/indexes/cluster.idx'

In [ ]:
with requests.get(cluster_url, stream=True) as r:
    with open(f'{current_idx["id"]}_cluster.idx', 'wb') as f:
        shutil.copyfileobj(r.raw, f)

In [ ]:
df = pd.read_table(f'{current_idx["id"]}_cluster.idx', header=None)

In [ ]:
df.head()

In [ ]:
au_indexes = list(pd.unique(df.loc[df[0].str.startswith('au,')][1]))

In [ ]:
for au_index in au_indexes:
    index_url = f'https://commoncrawl.s3.amazonaws.com/cc-index/collections/{current_idx["id"]}/indexes/{au_index}'
    with s.get(index_url, stream=True, timeout=60) as r:
        with open(au_index, 'wb') as f:
            shutil.copyfileobj(r.raw, f)

In [ ]:
for au_index in au_indexes:
    with gzip.open(au_index, 'rb') as f_in:
        with open(f'{au_index[:-3]}.txt', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)